##Installing Dependencies

In [1]:
!git lfs install
!git clone https://huggingface.co/google/mt5-base

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'mt5-base'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 27 (delta 1), reused 6 (delta 1), pack-reused 21
Unpacking objects: 100% (27/27), 2.23 MiB | 2.10 MiB/s, done.
Filtering content: 100% (3/3), 6.50 GiB | 154.68 MiB/s, done.


In [2]:
!pip install wandb -qqq
!pip install simpletransformers -q

import logging
import pandas as pd
from simpletransformers.t5 import T5Args
from simpletransformers.t5 import T5Model, T5Args
import torch
import numpy as np
import os
import random
import wandb


#For logging loss
wandb.login(key="46cb1e2ffa78177f23adbe2d7d16cf09a6176348")
from google.colab import drive
drive.mount('/content/drive')

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(7)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 108.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Mounted at /content/drive


#Data Preprocessing

In [3]:
import math

# specify input file name and two output file names
input_file1 = "/content/drive/MyDrive/Arabic.txt"
output_file1 = "train.ar"
output_file2 = "test.ar"
input_file2 = "/content/drive/MyDrive/Swahili.txt"
output_file3 = "train.sw"
output_file4 = "test.sw"
# specify the percentage of lines to be written to the second file
percent_second_file = 5  # change this value as needed

# calculate the number of lines to be written to the second file
with open(input_file1, 'r') as infile:
    num_lines = sum(1 for line in infile)
num_lines_second_file = math.ceil(num_lines * percent_second_file / 100)

# open input and output files
with open(input_file1, 'r') as infile, open(output_file1, 'w') as outfile1, open(output_file2, 'w') as outfile2:
  # iterate over input file lines and write to output files
  for i, line in enumerate(infile):
      if i < num_lines_second_file:
          outfile2.write(line)
      else:
          outfile1.write(line)





# calculate the number of lines to be written to the second file
with open(input_file2, 'r') as infile:
    num_lines = sum(1 for line in infile)
num_lines_second_file = math.ceil(num_lines * percent_second_file / 100)

# open input and output files
with open(input_file2, 'r') as infile, open(output_file3, 'w') as outfile1, open(output_file4, 'w') as outfile2:
    # iterate over input file lines and write to output files
    for i, line in enumerate(infile):
        if i < num_lines_second_file:
            outfile2.write(line)
        else:
            outfile1.write(line)

# display confirmation message
print(f"Input file '{input_file1}' has been divided into '{output_file1}' and '{output_file2}'.")
print(f"Input file '{input_file2}' has been divided into '{output_file3}' and '{output_file4}'.")


def prepare_translation_datasets(data_path):
    with open(os.path.join(data_path, "train.sw"), "r", encoding="utf-8") as f:
        sw_text = f.readlines()
        sw_text = [text.strip("\n") for text in sw_text]

    with open(os.path.join(data_path, "train.ar"), "r") as f:
        arabic_text = f.readlines()
        arabic_text = [text.strip("\n") for text in arabic_text]

    data = []
    for swahili, arabic in zip(sw_text, arabic_text):
        data.append(["translate swahili to arabic", swahili, arabic])
        data.append(["translate arabic to swahili", arabic, swahili])

    train_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])


    
    with open(os.path.join(data_path, "test.sw"), "r", encoding="utf-8") as f:
        sw_text = f.readlines()
        sw_text = [text.strip("\n") for text in sw_text]

    with open(os.path.join(data_path, "test.ar"), "r") as f:
        arabic_text = f.readlines()
        arabic_text = [text.strip("\n") for text in arabic_text]
    
    data = []
    for swahili, arabic in zip(sw_text, arabic_text):
        data.append(["translate swahili to arabic", swahili, arabic])
        data.append(["translate arabic to swahili", arabic, swahili])

    

    eval_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])

    return train_df, eval_df

train_df, eval_df = prepare_translation_datasets("/content")

train_df.to_csv("/content/train.tsv", sep="\t",index = False)
eval_df.to_csv("/content/eval.tsv", sep="\t", index= False)


Input file '/content/drive/MyDrive/Arabic.txt' has been divided into 'train.ar' and 'test.ar'.
Input file '/content/drive/MyDrive/Swahili.txt' has been divided into 'train.sw' and 'test.sw'.


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62342 entries, 0 to 62341
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   prefix       62342 non-null  object
 1   input_text   62342 non-null  object
 2   target_text  62342 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [5]:
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3282 entries, 0 to 3281
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   prefix       3282 non-null   object
 1   input_text   3282 non-null   object
 2   target_text  3282 non-null   object
dtypes: object(3)
memory usage: 77.0+ KB


#Training mT5 Model

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
import os
import logging
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args
from transformers import T5Config


train_df = pd.read_csv("/content/train.tsv", sep="\t").astype(str)
eval_df  = pd.read_csv("/content/eval.tsv", sep="\t").astype(str)

model_args = T5Args()
model_args.max_seq_length = 256
model_args.train_batch_size = 16
model_args.eval_batch_size = 16
model_args.num_train_epochs = 20
model_args.scheduler = "cosine_schedule_with_warmup"
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.learning_rate = 0.0003
model_args.optimizer = 'Adafactor'
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.save_model_every_epoch = True
model_args.use_early_stopping = True
model_args.num_return_sequences = 1
model_args.do_lower_case = True
model_args.output_dir = "/content/checkpoints"
model_args.best_model_dir = "/content/drive/MyDrive/mt5_trans_frezing/best_model"
model_args.wandb_project = "T5-base Translation using prefix final Layers"


# Load the pre-trained MT5 configuration
mt5_config = T5Config.from_pretrained("google/mt5-base")

# Freeze the first 4 encoder layers and the last 2 decoder layers
mt5_config.num_layers = 8  # Only use the last 8 encoder layers
mt5_config.num_decoder_layers = 10  # Only use the last 10 decoder layers

model_config = mt5_config

# model_output_dir = "/content/checkpoints/checkpoint-7794-epoch-2"
# Initialize the MT5 model with the modified configuration
model = T5Model(
    "mt5",  # Pass the model name as a string
    '/content/drive/MyDrive/mt5_trans_frezing/best_model',
    config=model_config, 
    args=model_args,
)

model.train_model(train_df, eval_data=eval_df)

    # cuda_devices=[0]  # optional, specify the GPU device to use
# Optional: Evaluate the model. We'll test it properly anyway.
# results = model.eval_model(eval_df, verbose=True)

# layers freezing

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


  0%|          | 0/62342 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3712: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprec

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

wandb: Currently logged in as: arabic-swahilimt (arabic-swahili). Use `wandb login --relogin` to force relogin


Running Epoch 0 of 20:   0%|          | 0/3897 [00:00<?, ?it/s]

#Evaluation (BLUE)

In [6]:
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 12.4 MB/s eta 0:00:00


In [7]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "6"
import logging
import sacrebleu
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args
import torch
torch.manual_seed(0)
import random 
random.seed(0)

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)



model_args = T5Args()
model_args.max_length = 128
model_args.length_penalty = 1
model_args.num_beams = 5
model_args.eval_batch_size = 32


model_output_dir = "/content/drive/MyDrive/mt5_trans_frezing/best_model"

model = T5Model("mt5", model_output_dir, args=model_args, )#cuda_devices=[6])

eval_df = pd.read_csv("/content/eval.tsv", sep="\t").astype(str)

ar_truth = [eval_df.loc[eval_df["prefix"] == "translate swahili to arabic"]["target_text"].tolist()]
to_arabic = eval_df.loc[eval_df["prefix"] == "translate swahili to arabic"]["input_text"].tolist()
ar_truth_list = eval_df.loc[eval_df["prefix"] == "translate swahili to arabic"]["target_text"].tolist()

swahili_truth = [eval_df.loc[eval_df["prefix"] == "translate arabic to swahili"]["target_text"].tolist()]
to_swahili = eval_df.loc[eval_df["prefix"] == "translate arabic to swahili"]["input_text"].tolist()
swahili_truth_list = eval_df.loc[eval_df["prefix"] == "translate arabic to swahili"]["target_text"].tolist()
sw2ar = "translate swahili to arabic: "
ar2sw = "translate arabic to swahili: "
to_arabic_ = [sw2ar + s for s in to_arabic]
to_swahili_  = [ar2sw + s for s in to_swahili]




# Predict
arabic_preds = model.predict(to_arabic_)


sw_ar_bleu = sacrebleu.corpus_bleu(arabic_preds, ar_truth)
print("--------------------------")
print("swahili to Arabic: ", sw_ar_bleu.score)

print(f"the type of the prediction is type(arabic_preds)")

with open("Swahili_to_Arabic_REPORT_RESULTS.txt", "w", encoding="utf-8") as fb:
    counter=0
    for index,ar in enumerate(arabic_preds):
        source_line = "src: " + to_arabic[counter] + "\n"
        fb.write(source_line)
        real_line = "real: " + str(ar_truth_list[counter]) + "\n"
        fb.write(real_line)
        pred_line = "pred: " + str(ar) +"\n"
        
        fb.write(pred_line)
        sperator = " --------------     --------------  ----------------   -------------  \n "
        fb.write(sperator)
        counter +=1 


swahili_preds = model.predict(to_swahili_)

ar_sw_bleu = sacrebleu.corpus_bleu(swahili_preds, swahili_truth)
print("Arabic to Swahili: ", ar_sw_bleu.score)

counter=0
with open("Arabic_to_Swahili_REPORT_RESULTS.txt", "w", encoding="utf-8") as fb:
    for index,sw in enumerate(swahili_preds):
        source_line =  "src: " + to_swahili[counter] + "\n"
        fb.write(source_line)
        real_line = "real: " + str(swahili_truth_list[counter]) + "\n"
        fb.write(real_line)
        pred_line = "pred: " + str(sw) +"\n"
        counter +=1
        fb.write(pred_line)
        sperator = " --------------     --------------  ----------------   -------------  \n "
        fb.write(sperator)

Generating outputs:   0%|          | 0/52 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3712: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/1641 [00:00<?, ?it/s]

--------------------------
swahili to Arabic:  1.2801501139195255
the type of the prediction is type(arabic_preds)


Generating outputs:   0%|          | 0/52 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1641 [00:00<?, ?it/s]

Arabic to Swahili:  2.151890253015057
